In [7]:
# pip install ortools
import pandas as pd
from ortools.constraint_solver import pywrapcp
from ortools.constraint_solver import routing_enums_pb2

def create_data_model():
    """Stores the data for the problem."""
    from scipy.spatial import distance_matrix
    global today_inspections
    
    # Import inspection locations
    csv_path = "inspection_locations.csv"  # This is located in the same directory
    inspection_locations = pd.read_csv(csv_path)

    # Create a dataframe with one line pre-populated for the health department
    data = [['Health Department',40.6533382,-111.8720126,'788 Woodoak Ln', 'Murray', 'UT', 84107]]
    location = [0]
    today_inspections = pd.DataFrame(data, columns=['name', 'lat', 'lng', 'address', 'city', 'state', 'postal_code'], index=location)

    # Add a specified number of inspection_locations for today's inspections (this would normally be done by due date)
    num_inspections = 8
    today_inspections = today_inspections.append(inspection_locations.sample(num_inspections, random_state=123), ignore_index=True)

    # Create distance matrix for today's inspections
    distance_matrix =  pd.DataFrame(distance_matrix(today_inspections.loc[:,('lat','lng')].to_numpy(), today_inspections.loc[:,('lat','lng')].to_numpy()),
                                    index=today_inspections.index, columns=today_inspections.index)

    # Scale up the distance matrix
    distance_matrix = distance_matrix*10_000
    distance_matrix = distance_matrix.values.tolist()
    
    # Put it into the format that OR-Tools expects
    data = {}
    data['distance_matrix'] = distance_matrix
    data['num_vehicles'] = 1  # Only one vehicle for this problem
    data['depot'] = 0  # The depot is index 0 in the distance matrix
    return data

def print_solution(manager, routing, solution):
    """Prints solution on console."""
    # print('Objective: {} miles'.format(solution.ObjectiveValue()))  # Need to adjust this for when the distance matrix is scaled
    index = routing.Start(0)
    plan_output = 'Route for vehicle 0:\n'
    route_distance = 0
    while not routing.IsEnd(index):
        plan_output += ' {} ->'.format(manager.IndexToNode(index))
        previous_index = index
        index = solution.Value(routing.NextVar(index))
        route_distance += routing.GetArcCostForVehicle(previous_index, index, 0)
    plan_output += ' {}\n'.format(manager.IndexToNode(index))
    print(plan_output)
    plan_output += 'Route distance: {}miles\n'.format(route_distance)

def get_routes(solution, routing, manager):
    """Get vehicle routes from a solution and store them in an array."""
    # Get vehicle routes and store them in a two dimensional array whose
    # i,j entry is the jth location visited by vehicle i along its route.
    routes = []
    for route_nbr in range(routing.vehicles()):
        index = routing.Start(route_nbr)
        route = [manager.IndexToNode(index)]
        while not routing.IsEnd(index):
            index = solution.Value(routing.NextVar(index))
            route.append(manager.IndexToNode(index))
        routes.append(route)
    return routes

def main():
    """This makes everything happen."""
    
    data = create_data_model()
    
    # Create the routing index manager
    manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']), data['num_vehicles'], data['depot'])

    # Creates the routing model
    routing = pywrapcp.RoutingModel(manager)


    def distance_callback(from_index, to_index):
        """Returns the distance between the two nodes."""
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data['distance_matrix'][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)

    # This defines the cost of travel.  In this case it is just the distance.
    # This could be modified to account for different vehicle types or other costs, such as tolls.
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    # This search parameter finds the first solution that works
    # PATH_CHEAPEST_ARC just uses the least expensive route available that doesn't lead to a previously visited node
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)

    # # This search parameter takes some extra time to explore outside of any local minima it may encounter
    # search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    # search_parameters.local_search_metaheuristic = (routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH)
    # search_parameters.time_limit.seconds = 30
    # search_parameters.log_search = True

    # Print solution for the optimal route
    solution = routing.SolveWithParameters(search_parameters)
    if solution:
        print_solution(manager, routing, solution)
    
    # Create ordered list of inspections by index (route)
    routes = get_routes(solution, routing, manager)
    for i, route in enumerate(routes):
      route
    
    # Sort today_inspections by the recommended route
    ordered_inspections = today_inspections.iloc[route]
    print(ordered_inspections)
    
if __name__ == '__main__':
    main()

Route for vehicle 0:
 0 -> 7 -> 8 -> 6 -> 4 -> 5 -> 3 -> 1 -> 2 -> 0

                       name        lat         lng             address  \
0         Health Department  40.653338 -111.872013      788 Woodoak Ln   
7                      Buds  40.763001 -111.876332         509 E 300 S   
8   Even Stevens Sandwiches  40.764636 -111.879047  200 South 414 East   
6                       Eva  40.762268 -111.890847       317 S Main St   
4  Siegfried's Delicatessen  40.765035 -111.892060          20 W 200 S   
5                  Valter's  40.762722 -111.896216      173 W Broadway   
3    Alberto's Mexican Food  40.758170 -111.899092         511 S 300 W   
1                   R&R BBQ  40.755812 -111.900033         307 W 600 S   
2            Tacos Don Rafa  40.752262 -111.888470     799 S State St.   
0         Health Department  40.653338 -111.872013      788 Woodoak Ln   

             city state  postal_code  
0          Murray    UT        84107  
7  Salt Lake City    UT        84102 